In [1]:
from langchain_openai import ChatOpenAI
import pandas as pd
from pyprojroot import here
import os
from dotenv import load_dotenv
from sqlalchemy import create_engine, inspect
from langchain_community.utilities import SQLDatabase
from langchain_community.agent_toolkits import create_sql_agent


import warnings
warnings.filterwarnings("ignore")
OPEN_AI_API_KEY = os.getenv("OPEN_AI_API_KEY")
os.environ["LANGCHAIN_API_KEY"]= os.getenv("LANGCHAIN_API_KEY")
os.environ["LANGCHAIN_TRACING_V2"]= os.getenv("LANGCHAIN_TRACING_V2")
os.environ["LANGCHAIN_PROJECT"]=os.getenv("LANGCHAIN_PROJECT")

### **SQL**

Using SQL to interact with CSV data is the recommended approach because it is easier to limit permissions and sanitize queries than with arbitrary Python.

Most SQL databases make it easy to load a CSV file in as a table (DuckDB, SQLite, etc.). Once you’ve done this you can use all of the chain and agent-creating techniques outlined in the SQL use case guide. Here’s a quick example of how we might do this with SQLite:

In [2]:
db_path="all_data.db"
engine = create_engine(f"sqlite:///{db_path}")
connection = engine.connect()
inspector = inspect(engine)
table_names = inspector.get_table_names()
table_names

['data_restaurants']

In [3]:
db_path="all_data.db"
engine = create_engine(f"sqlite:///{db_path}")
connection = engine.connect()
inspector = inspect(engine)
table_names = inspector.get_table_names()
db= SQLDatabase(engine=engine)
llm = ChatOpenAI(
    model="gpt-3.5-turbo",
    temperature=1,
    model_name="amiguis")
agent_executor = create_sql_agent(llm, db=db, agent_type="openai-tools", verbose=True)

In [4]:
agent_executor.invoke({"input": "What is the best restaurant neighborhood in the la latina neighborhood, infer the neighborhood in the vicinity column where has the addres andgeometry.location.lat geometry.location.long where are the coordinates in the vicinity column there is the name of a street, you as an ai pe trained model match the name of that street with the data from your training process and infer the neighborhood and district ?"})



> Entering new SQL Agent Executor chain...

Invoking: `sql_db_list_tables` with `{}`


data_restaurants
Invoking: `sql_db_schema` with `{'table_names': 'data_restaurants'}`



CREATE TABLE data_restaurants (
	curbside_pickup INTEGER, 
	delivery INTEGER, 
	dine_in INTEGER, 
	formatted_phone_number TEXT, 
	name TEXT, 
	open_now INTEGER, 
	periods TEXT, 
	weekday_text TEXT, 
	place_id TEXT, 
	price_level REAL, 
	rating REAL, 
	reviews TEXT, 
	serves_beer INTEGER, 
	serves_breakfast INTEGER, 
	serves_brunch INTEGER, 
	serves_dinner INTEGER, 
	serves_lunch INTEGER, 
	serves_vegetarian_food INTEGER, 
	serves_wine INTEGER, 
	takeout INTEGER, 
	url TEXT, 
	user_ratings_total REAL, 
	website TEXT, 
	wheelchair_accessible_entrance INTEGER, 
	reservable INTEGER, 
	overview TEXT, 
	error_message TEXT, 
	types TEXT, 
	vicinity TEXT, 
	"geometry.location.lat" REAL, 
	"geometry.location.lng" REAL
)

/*
3 rows from data_restaurants table:
curbside_pickup	delivery	dine_in	formatted_phone_number	name	

{'input': 'What is the best restaurant neighborhood in the la latina neighborhood, infer the neighborhood in the vicinity column where has the addres andgeometry.location.lat geometry.location.long where are the coordinates in the vicinity column there is the name of a street, you as an ai pe trained model match the name of that street with the data from your training process and infer the neighborhood and district ?',
 'output': 'Based on the data from the `data_restaurants` table, I can see information about three restaurants located in Madrid. Here are some of the details for the restaurants in the La Latina neighborhood:\n\n1. Name: Café & Té\n   - Address: Calle de Goya, 18, Madrid\n   - Coordinates: Latitude 40.4251, Longitude -3.6839\n   - Rating: 3.5\n\n2. Name: chök - Goya (Gluten Free)\n   - Address: Calle de Goya, 49, Madrid\n   - Coordinates: Latitude 40.4253, Longitude -3.6821\n   - Rating: 3.7\n\n3. Name: Sushiaoi\n   - Address: Calle de Núñez de Balboa, 37, Madrid\n   - 